# Importing all required libraries

In [16]:
from PyPDF2 import PdfReader
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Extract Text from Resume (PDF)

In [17]:
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Now call the function with your file path
resume_text = extract_text_from_pdf("/content/Himanshu_Data_Analyst_Resume.pdf")

# Extract Skills using a Hugging Face model

In [18]:
def extract_skills(text):

    ner = pipeline("ner", model="yashpwr/resume-ner-bert-v2", aggregation_strategy="simple")
    entities = ner(text)
    skills = [ent["word"] for ent in entities if ent["entity_group"] == "SKILL"]
    extracted_skills = list(set(skills))
    data_analyst_skills = [
        "Python", "R", "SQL", "Excel", "Power BI", "Tableau", "Google Sheets", "Jupyter Notebook",
        "Pandas", "NumPy", "Matplotlib", "Seaborn", "Scikit-learn", "Statsmodels",
        "ETL", "Data Cleaning", "Data Wrangling", "Data Transformation", "Data Visualization",
        "Data Modeling", "Data Integration", "PostgreSQL", "MySQL", "Snowflake", "BigQuery",
        "Azure Data Studio", "AWS S3", "KPI Tracking", "Dashboard Development",
        "Predictive Modeling", "Regression Analysis", "Time Series Forecasting",
        "Statistical Analysis", "A/B Testing", "Exploratory Data Analysis", "Power Query",
        "DAX", "SQLAlchemy", "Automation", "Workflow Optimization", "Classification",
        "Clustering", "Decision Trees", "Random Forest", "Feature Engineering",
        "Model Evaluation", "Business Intelligence", "Reporting", "Data-Driven Decision Making",
        "Critical Thinking", "Problem Solving"
    ]

    extracted_lower = [s.lower() for s in extracted_skills]
    expected_lower = [s.lower() for s in data_analyst_skills]

    matched = [s for s in expected_lower if s in extracted_lower]
    missing = [s for s in expected_lower if s not in extracted_lower]
    match_percent = (len(matched) / len(expected_lower)) * 100



# Compare Resume with Job Description using Embeddings

In [20]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

jd_text = """
We are looking for a Data Analyst skilled in SQL, Power BI, and Python for building dashboards and predictive analytics models.
"""
emb1 = model.encode(resume_text, convert_to_tensor=True)
emb2 = model.encode(jd_text, convert_to_tensor=True)
similarity = util.pytorch_cos_sim(emb1, emb2)
print("Resume–JD Similarity Score:", similarity.item())


Resume–JD Similarity Score: 0.647477388381958


# Generate Feedback using Hugging Face LLM

In [23]:
generator = pipeline("text-generation", model="HuggingFaceTB/SmolLM3-3B", trust_remote_code=True)
prompt = f"""
You are an AI resume reviewer.
Analyze the following resume and job description.
Provide feedback in bullet points:
- Resume Strength
- Missing Keywords
- Suggestions for improvement

Resume:
{resume_text[:1000]}  # first 1000 characters to avoid long input

Job Description:
{jd_text}
"""
response = generator(prompt, max_new_tokens=1000)
print(response[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



You are an AI resume reviewer.
Analyze the following resume and job description.
Provide feedback in bullet points:
- Resume Strength
- Missing Keywords
- Suggestions for improvement

Resume:
HIMANSHU SIROHI  
Data Analyst  | Business Analyst  
 New Delhi, Indi a | +91-969090067 6 | LinkedIn  | GitHub  | Himanshusirohi41@gmail.com  
 
 
EDUCATION  
Bachelor  of Science  in Mathematics  | M.J.P  Rohilkhand  University  | Bareilly,  Uttar Pradesh  — June 2017  
 
PROFESSIONAL SUMMARY  
Data Analyst  with hands -on experience in  Python (Pandas, NumPy, Matplotlib, Seaborn), SQL, and Power BI . 
Skilled in data cleaning, reporting , and transforming raw datasets into strategic insights for business growth. 
Experienced in forecasting , KPI tracking , statistical analysis , and interactive dashboard development . Strong 
communicator with a proven ability to collaborate across teams and align data strategies with business goals.  
 
TECHNICAL SKILLS  
Programming and Analytics: Python (Pan